[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MaxMitre/DeepLearning/blob/main/Semana04_Q_Learning.ipynb)

# Problema

Se resolverá un problema de optimización, más específicamente, un problema sobre políticas de mantenimiento en una línea de producción y esto se llevará a cabo utilizando Q-learning (aprendizaje por refuerzo)



# Dependencias

In [ ]:
import pandas as pd
import numpy as np
import random
import math

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

plt.rcParams['axes.unicode_minus'] = False

In [ ]:
# Etapa de entrada, Recompensa devuelta y estado de la siguiente etapa
def get_enviroment(state, action):

    # El estado se compone de 3 entradas: Etapa, Cantidad en inventario, Duración (si es positiva, es un tiempo de mantenimiento)
    stage = state[0]
    inventory_level = state[1]
    duration = state[2]

    print("current_stage: ", stage)
    print("current_inventory_level: ", inventory_level)
    print("current_duration: ", duration)

    # Dos factores aleatorios, el que tanto se daña una máquina con el uso y que tanto producto es encargado
    random_rate = random.random()
    # DEMANDA de clientes
    order_num = np.max(int(random.gauss(mu=MU,sigma=SIG)), 0)
    print("order_num",order_num)

    # Tenemos 5 posibles acciones
    # 0: Se sigue produciendo
    # 1: Mantenimiento preventivo (PM)
    # 2: Mantenimiento Correctivo (CM)
    # 3: En mantenimiento (Hay que esperar mas etapas)
    # 4: Apagar la máquina (Rest)
    if action == 0:
        print("keep producing")
        new_stage = stage + 1 if MACHINE_DETERIORATION_RATE[stage]>random_rate else stage
        new_inventory_level = inventory_level + int(MACHINE_YIELD_RATE[new_stage] * MACHINE_PRODUCTION_NUM) - order_num
        new_duration = 0
    elif action == 1:
        print("preventive maintenance")
        new_stage = stage - 2 if stage > 2 else 0
        new_inventory_level = inventory_level - order_num
        new_duration = PM_DURATION
    elif action == 2:
        print("corrective maintenance")
        new_stage = 0
        new_inventory_level = inventory_level - order_num
        new_duration = CM_DURATION
    elif action==3:
        print("under maintenance")
        new_stage = stage
        new_inventory_level = inventory_level - order_num
        new_duration = duration - 1
    else:
        print("rest")
        new_stage = stage
        new_inventory_level = inventory_level - order_num
        new_duration = 0


    # "recompensa" y verifica si "new_inventory_level" está fuera de los límites
    old_reward = inventory_level * BUFFER_COST if inventory_level >=0 else abs(inventory_level) * BACKORDER_COST

    if new_inventory_level >= INVENTORY_LEVEL/2:
        new_inventory_level = INVENTORY_LEVEL/2
    elif new_inventory_level <= -(INVENTORY_LEVEL/2):
        new_inventory_level = -(INVENTORY_LEVEL/2)

    reward = new_inventory_level * BUFFER_COST if new_inventory_level >=0 else abs(new_inventory_level) * BACKORDER_COST

    print("--after action--")
    print("next_stage: ", new_stage)
    print("next_inventory_level: ", new_inventory_level)
    print("next_duration: ", new_duration)
    return  (new_stage, new_inventory_level, new_duration), reward



In [ ]:
def choose_action(state, q_table, epsilon):
    stage = state[0]
    inventory_level = state[1]
    duration = state[2]

    random_prob = np.random.random_sample()
    if duration > 0: # Suponiendo que todavía está en mantenimiento, devuelva el mantenimiento directamente
        # La máquina está en mantenimiento, acción: ESPERAR
        return 3
    elif stage == 5: # Suponiendo que la etapa haya llegado a 5, seleccione directamente la acción de CM
        # acción: MANTENIMIENTO CORRECTIVO
        return 2

    elif stage < 1: # Si el escenario es completamente nuevo, no habrá PM, solo fabricación o apagado.
        if random_prob < epsilon:
            print('exploración')
            rt = np.random.random_sample()
            if rt < 1/2:
                return 0 # Produciendo
            else:
                return 4 # Apagada
        else:
            print('explotación')
            produce_value = q_table.loc[state, 0]
            rest_value = q_table.loc[state, 4]
            if produce_value >= rest_value:
                return 0 # Produciendo
            else:
                return 4 # Apagada
    else: # Cuando la etapa >= 1 hay 3 posibilidades: PM, fabricación, se puede apagar
        if random_prob < epsilon:
            print('exploración')
            rt = np.random.random_sample()
            if rt < 1/3:
                return 0
            elif 2/3 >= rt >= 1/3:
                return 1
            else:
                print("Apagar")
                return 4
        else:
            # explotación
            produce_value = q_table.loc[state, 0]
            pm_value = q_table.loc[state, 1]
            rest_value = q_table.loc[state, 4]

            if produce_value >= pm_value and produce_value >= rest_value:
                return 0 # Producir
            elif pm_value > produce_value and pm_value > rest_value:
                return 1 # PM
            else:
                return 4 # Apagar


El Tensor "q_table" tiene 3 índices:
- El primero representa el estado de la máquina
- El segundo el estado del inventario
- El tercero duración de la máquina (si es positiva, tiempo de mantenimiento)

Al seleccionar un sub-DataFrame de "q_table" es la matriz de transición dado el estado con el que indexemos.

In [ ]:
# estado de la máquina
STAGE = 6 # Etapas de vida de la máquina
MACHINE_DETERIORATION_RATE = []
MACHINE_YIELD_RATE = [1.0, 0.8, 0.6, 0.4, 0.2, 0]
MACHINE_PRODUCTION_NUM = 10

# relacionado con el costo
INVENTORY_LEVEL = 80
BUFFER_COST = -3
BACKORDER_COST = -5

# relacionado con el MANTENIMIENTO
PM_DURATION = 1
CM_DURATION = 3
DURATION = 3

# Parámetros para la demanda, pueden complicarlo un poco si consideran esta variable como POISSON
MU = 5
SIG = 1

lambd = 0.2
for i in range(1, 6):
    j = 1 - np.exp(-lambd*i)
    MACHINE_DETERIORATION_RATE.append(j)
MACHINE_DETERIORATION_RATE.append(0)

In [ ]:
MACHINE_DETERIORATION_RATE

In [ ]:
state, action_1, action_2, action_3, action_4, action_5 = [], [], [], [], [], []

for i in range(STAGE):
    for j in range((INVENTORY_LEVEL//2) * -1, (INVENTORY_LEVEL//2)+1):
            for k in range(DURATION+1):
                state.append((i,j,k))
                action_1.append(float("-1000"))
                action_2.append(float("-1000"))
                action_3.append(float("-1000"))
                action_4.append(float("-1000"))
                action_5.append(float("-1000"))
# 0: produce, 1:pm, 2:cm, 3:maintenance
q_table = pd.DataFrame({0:action_1, 1:action_2, 2:action_3,3:action_4, 4:action_5}, index=pd.MultiIndex.from_tuples(state))
initial_state = (0, 1, -4)
#q_table.loc[initial_state]
q_table

In [ ]:
# El primero representa el estado de la máquina
# El segundo el estado del inventario
# El tercero duración de la máquina (si es positiva, tiempo de mantenimiento)
q_table.loc[(4,3,0),:]

Los cambios en las matrices Q ocurren del siguiente modo:

<center><img src='https://camo.githubusercontent.com/f785f991b1eae6c736d51327ecd2b8f354d2a985d19b1e91b3191c52eec68615/68747470733a2f2f692e696d6775722e636f6d2f4f587638654d682e706e67' width=100000></center>



In [ ]:
from tqdm import tqdm

In [ ]:
# Diseño paramétrico

state = (0,10,0)
alpha = 0.5
dis_fac = 0.8
get_epsilon = lambda i: max(0.1, min(1, 1.0 - math.log10((i+1)/2500)))

reward_list = []
q_value = []
avg_reward_list = []
total_reward = 0
inventory_level_list = []
total_inventory = 0
avg_inventory_list = []

for i in tqdm(range(50000)):

    q_value.append(sum(q_table.sum()))
    epsilon = get_epsilon(i)
#     epsilon = 0
    inventory_level_list.append(state[1])

    print("###############iter", i,"###############")
    action = choose_action(state, q_table, epsilon)
    new_state, reward = get_enviroment(state, action)
    # Recordemos que el estado se compone de (etapa, cantidad de inventario, duración)

    print("$$$$$", reward, "$$$$$$$$$$")

    # El estado que no ha cambiado en algunos enlaces solo puede tener acciones específicas


    future_value = float('-inf')
    for j in range(5):
        value = q_table.loc[new_state, j]
        if value > future_value:
            future_value = value

    if new_state[2] > 0:
        future_value = q_table.loc[new_state, 3]
    if new_state[2] == CM_DURATION:
        future_value = q_table.loc[new_state, 2]

# 0: produce, 1:pm, 2:cm, 3:maintenance 4: apagar

    # Actualización de valores de Q
    q_table.loc[state, action] = (1-alpha) * q_table.loc[state, action] + alpha * (reward + dis_fac *future_value)

    total_reward += reward
    total_inventory += state[1]
    avg_inventory_list.append(total_inventory/(i+1))
    avg = total_reward/(i+1)
    if i > 5000:
        if sum(avg_reward_list[-50:])/len(avg_reward_list[-50:]) == avg:
            print("early stop")
            break

    avg_reward_list.append(avg)
    state = new_state

In [ ]:
plt.xlabel("iteration")
plt.ylabel("sum of q_table")
plt.title("sum of q_table")
plt.plot(q_value)

In [ ]:
q_table

In [ ]:
plt.xlabel("iteration")
plt.ylabel("reward")
plt.title("avg reward")
plt.plot(avg_reward_list)

In [ ]:
plt.xlabel("iteration")
plt.ylabel("inventory_level")
plt.title("inventory_level")
plt.plot(inventory_level_list)

In [ ]:
plt.xlabel("iteration")
plt.ylabel("inventory_level")
plt.title("avg_inventory_level")
plt.plot(avg_inventory_list)

In [ ]:
q_table.shape

In [ ]:
q_table.loc[(1,10,0),:]

In [ ]:
q_table.loc[(0,-12,0),:]

In [ ]:
# Dado el nivel de inventario, veremos que da mas recompensa:
# Azul: Es mejor seguir produciendo
# Rojo: Es mejor dar mantenimiento preventivo (PM)
# Naranja: Apagar la máquina

x = np.arange(0,6)
y = np.arange(-40,41)
fig, ax = plt.subplots()
red_patch = mpatches.Patch(color='red', label='pm')
blue_patch = mpatches.Patch(color='blue', label='production')
orange_patch = mpatches.Patch(color='orange', label='rest')
#ax.legend(handles=[orange_patch, red_patch, blue_patch])

for i in range(INVENTORY_LEVEL//2 * -1, INVENTORY_LEVEL//2+1):
    for j in range(STAGE-1):

        if j >= 1:
            production_value = q_table.loc[(j, i, 0), 0]
            pm_value = q_table.loc[(j, i, 0), 1]
            rest_value = q_table.loc[(j,i, 0), 4]

            if max(production_value, pm_value, rest_value) == production_value:
                ax.scatter(j,i, c='blue', s=5)
            elif max(production_value, pm_value, rest_value) == pm_value:
                ax.scatter(j,i,c='red',s=5)
            elif max(production_value, pm_value, rest_value) == rest_value:
                ax.scatter(j,i,c='orange', s=5)
            else:
                continue
        else:
            production_value = q_table.loc[(j, i, 0), 0]
            rest_value = q_table.loc[(j, i, 0), 4]
            if production_value > rest_value:
                ax.scatter(j,i, c='blue', s=5)
            elif production_value < rest_value:
                ax.scatter(j,i,c='orange', s=5)
            else:
                continue

plt.xlabel("machine_age_stage")
plt.ylabel("inventory_level")
plt.title("pm_policy")
plt.show()


In [ ]:
# Azul: Es mejor seguir produciendo
# Rojo: Es mejor dar mantenimiento preventivo (PM)
# Naranja: Apagar la máquina

# Referencias

[Reinforcement Learning: Preventive Maintenance](https://github.com/jerryshen1216/ORA_Project_RL_PM)